In [1]:
%env WANDB_PROJECT=PII
%env WANDB_RUN_GROUP=BiLSTM-longformer-3072-filter+T-CE

env: WANDB_PROJECT=PII
env: WANDB_RUN_GROUP=BiLSTM-longformer-3072-filter+T-CE


In [3]:
import json
import copy
import gc
import os
import re
from collections import defaultdict
from pathlib import Path

import torch
from torch import Tensor, nn
import numpy as np
import pandas as pd
from spacy.lang.en import English
from transformers.tokenization_utils import PreTrainedTokenizerBase
from transformers.models.longformer import LongformerForTokenClassification, LongformerTokenizerFast
from transformers.trainer import Trainer
from transformers.training_args import TrainingArguments
from transformers.trainer_utils import EvalPrediction
from transformers.data.data_collator import DataCollatorForTokenClassification
from datasets import Dataset, DatasetDict, concatenate_datasets
import wandb

In [4]:
wandb.login(key=WANB_KEY)

wandb: Currently logged in as: emiz6413. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
DATA_DIR = Path("../dataset/")
OUTPUT_DIR = "output"
Path(OUTPUT_DIR).mkdir(exist_ok=True)

In [6]:
TRAINING_MODEL_PATH = "allenai/longformer-base-4096"
# TRAINING_MODEL_PATH = "hf-internal-testing/tiny-random-longformer"
TRAINING_MAX_LENGTH = 3072 if "tiny-random" not in TRAINING_MODEL_PATH else 512
EVAL_MAX_LENGTH = 3072 if "tiny-random" not in TRAINING_MODEL_PATH else 512
CONF_THRESH = 0.9
LR = 2.5e-5  # 1.5e-5 ~ 3e-5 for base # 5e-6 ~ 1e-5 for large
LR_SCHEDULER_TYPE = "linear"
NUM_EPOCHS = 3 if "tiny-random" not in TRAINING_MODEL_PATH else 0.1
BATCH_SIZE = 8
EVAL_BATCH_SIZE = 8
GRAD_ACCUMULATION_STEPS = 16 // BATCH_SIZE
WARMUP_RATIO = 0.1
WEIGHT_DECAY = 0.01
FREEZE_EMBEDDING = False
FREEZE_LAYERS = 0
GAMMA = 0
MASK_P = 0
# training data
N_SPLITS = 4
FILTER_ORIGINAL = True
MOTH = False
PJMATHMATICIAN = False
NICHOLAS = False
MPWARE = False
TONYAROBERTSON = True

In [7]:
args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    fp16=torch.cuda.is_available(),
    learning_rate=LR,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=EVAL_BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUMULATION_STEPS,
    report_to="wandb",
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=50,
    save_total_limit=1,
    logging_steps=10,
    metric_for_best_model="f5",
    greater_is_better=True,
    load_best_model_at_end=True,
    overwrite_output_dir=True,
    lr_scheduler_type=LR_SCHEDULER_TYPE,
    warmup_ratio=WARMUP_RATIO,
    weight_decay=WEIGHT_DECAY,
)

## Load dataset

In [8]:
with DATA_DIR.joinpath("train.json").open("r") as f:
    original_data = json.load(f)

extra_data = []  #

if MOTH:
    with DATA_DIR.joinpath("pii_dataset_fixed.json").open("r") as f:
        external = json.load(f)
    print("Moth's datapoints: ", len(external))
    extra_data.extend(external)

if PJMATHMATICIAN:
    with DATA_DIR.joinpath("moredata_dataset_fixed.json").open("r") as f:
        external = json.load(f)
    print("PJMathmatician's datapoints: ", len(external))
    extra_data.extend(external)

if NICHOLAS:
    with DATA_DIR.joinpath("mixtral-8x7b-v1.json").open("r") as f:
        external = json.load(f)
    print("Nicholas' datapoints: ", len(external))
    extra_data.extend(external)

if MPWARE:
    with DATA_DIR.joinpath("mpware_mixtral8x7b_v1.1.json").open("r") as f:
        external = json.load(f)
    print("MPWARE's datapoints: ", len(external))
    extra_data.extend(external)

if TONYAROBERTSON:
    with DATA_DIR.joinpath("Fake_data_1850_218.json").open("r") as f:
        external = json.load(f)
    print("tonyarobertson's datapoints: ", len(external))
    extra_data.extend(external)

print(f"len(extra_data): {len(extra_data)}")

tonyarobertson's datapoints:  1850
len(extra_data): 1850


In [9]:
all_labels = [
    'B-EMAIL', 'B-ID_NUM', 'B-NAME_STUDENT', 'B-PHONE_NUM', 'B-STREET_ADDRESS', 'B-URL_PERSONAL', 'B-USERNAME', 'I-ID_NUM', 'I-NAME_STUDENT', 'I-PHONE_NUM', 'I-STREET_ADDRESS', 'I-URL_PERSONAL', 'O'
]
id2label = {i: l for i, l in enumerate(all_labels)}
label2id = {v: k for k, v in id2label.items()}
target = [l for l in all_labels if l != "O"]

## Tokenization

In [10]:
class CustomTokenizer:
    def __init__(self, tokenizer: PreTrainedTokenizerBase, label2id: dict, max_length: int) -> None:
        self.tokenizer = tokenizer
        self.label2id = label2id
        self.max_length = max_length

    def __call__(self, example: dict) -> dict:
        # rebuild text from tokens
        text, labels, token_map = [], [], []

        for idx, (t, l, ws) in enumerate(
            zip(example["tokens"], example["provided_labels"], example["trailing_whitespace"])
        ):
            text.append(t)
            labels.extend([l] * len(t))
            token_map.extend([idx]*len(t))

            if ws:
                text.append(" ")
                labels.append("O")
                token_map.append(-1)

        text = "".join(text)
        labels = np.array(labels)

        # actual tokenization
        tokenized = self.tokenizer(
            "".join(text),
            return_offsets_mapping=True,
            truncation=True,
            max_length=self.max_length
        )

        token_labels = []

        for start_idx, end_idx in tokenized.offset_mapping:
            # CLS token
            if start_idx == 0 and end_idx == 0:
                token_labels.append(self.label2id["O"])
                continue

            # case when token starts with whitespace
            if text[start_idx].isspace():
                start_idx += 1

            try:
                token_labels.append(self.label2id[labels[start_idx]])
            except:
                continue

        length = len(tokenized.input_ids)

        return {**tokenized, "labels": token_labels, "length": length, "token_map": token_map}

## Instanciate the dataset

In [11]:
tokenizer = LongformerTokenizerFast.from_pretrained(TRAINING_MODEL_PATH)
train_encoder = CustomTokenizer(tokenizer=tokenizer, label2id=label2id, max_length=TRAINING_MAX_LENGTH)
eval_encoder = CustomTokenizer(tokenizer=tokenizer, label2id=label2id, max_length=EVAL_MAX_LENGTH)

ds = DatasetDict()

for key, data in zip(["original", "extra"], [original_data, extra_data]):
    ds[key] = Dataset.from_dict({
        "full_text": [x["full_text"] for x in data],
        "document": [str(x["document"]) for x in data],
        "tokens": [x["tokens"] for x in data],
        "trailing_whitespace": [x["trailing_whitespace"] for x in data],
        "provided_labels": [x["labels"] for x in data],
    })

## Metrics

In [12]:
def find_span(target: list[str], document: list[str]) -> list[list[int]]:
    idx = 0
    spans = []
    span = []

    for i, token in enumerate(document):
        if token != target[idx]:
            idx = 0
            span = []
            continue
        span.append(i)
        idx += 1
        if idx == len(target):
            spans.append(span)
            span = []
            idx = 0
            continue

    return spans


class PRFScore:
    """A precision / recall / F score."""

    def __init__(
        self,
        *,
        tp: int = 0,
        fp: int = 0,
        fn: int = 0,
    ) -> None:
        self.tp = tp
        self.fp = fp
        self.fn = fn

    def __len__(self) -> int:
        return self.tp + self.fp + self.fn

    def __iadd__(self, other):  # in-place add
        self.tp += other.tp
        self.fp += other.fp
        self.fn += other.fn
        return self

    def __add__(self, other):
        return PRFScore(
            tp=self.tp + other.tp, fp=self.fp + other.fp, fn=self.fn + other.fn
        )

    def score_set(self, cand: set, gold: set) -> None:
        self.tp += len(cand.intersection(gold))
        self.fp += len(cand - gold)
        self.fn += len(gold - cand)

    @property
    def precision(self) -> float:
        return self.tp / (self.tp + self.fp + 1e-100)

    @property
    def recall(self) -> float:
        return self.tp / (self.tp + self.fn + 1e-100)

    @property
    def f1(self) -> float:
        p = self.precision
        r = self.recall
        return 2 * ((p * r) / (p + r + 1e-100))

    @property
    def f5(self) -> float:
        beta = 5
        p = self.precision
        r = self.recall

        fbeta = (1+(beta**2))*p*r / ((beta**2)*p + r + 1e-100)
        return fbeta

    def to_dict(self) -> dict[str, float]:
        return {"p": self.precision, "r": self.recall, "f5": self.f5}


class MetricsComputerV2:
    nlp = English()

    def __init__(self, eval_ds: Dataset, label2id: dict, conf_thresh: float = 0.9) -> None:
        self.ds = eval_ds.remove_columns("labels").rename_columns({"provided_labels": "labels"})
        self.gt_df = self.create_gt_df(self.ds)
        self.label2id = label2id
        self.confth = conf_thresh
        self._search_gt()

    def __call__(self, eval_preds: EvalPrediction) -> dict:
        pred_df = self.create_pred_df(eval_preds.predictions)
        return self.compute_metrics_from_df(self.gt_df, pred_df)

    def _search_gt(self) -> None:
        email_regex = re.compile(r'[\w.+-]+@[\w-]+\.[\w.-]+')
        phone_num_regex = re.compile(r"(\(\d{3}\)\d{3}\-\d{4}\w*|\d{3}\.\d{3}\.\d{4})\s")
        self.emails = []
        self.phone_nums = []

        for _data in self.ds:
            # email
            for token_idx, token in enumerate(_data["tokens"]):
                if re.fullmatch(email_regex, token) is not None:
                    self.emails.append(
                        {"document": _data["document"], "token": token_idx, "label": "B-EMAIL", "token_str": token}
                    )
            # phone number
            matches = phone_num_regex.findall(_data["full_text"])
            if not matches:
                continue
            for match in matches:
                target = [t.text for t in self.nlp.tokenizer(match)]
                matched_spans = find_span(target, _data["tokens"])
            for matched_span in matched_spans:
                for intermediate, token_idx in enumerate(matched_span):
                    prefix = "I" if intermediate else "B"
                    self.phone_nums.append(
                        {"document": _data["document"], "token": token_idx, "label": f"{prefix}-PHONE_NUM", "token_str": _data["tokens"][token_idx]}
                    )

    @staticmethod
    def create_gt_df(ds: Dataset):
        gt = []
        for row in ds:
            for token_idx, (token, label) in enumerate(zip(row["tokens"], row["labels"])):
                if label == "O":
                    continue
                gt.append(
                    {"document": row["document"], "token": token_idx, "label": label, "token_str": token}
                )
        gt_df = pd.DataFrame(gt)
        gt_df["row_id"] = gt_df.index

        return gt_df

    def create_pred_df(self, prediction: np.ndarray) -> pd.DataFrame:
        ### construct prediction df
        o_index = self.label2id["O"]
        preds = prediction.argmax(-1)
        preds_without_o = prediction[:,:,:o_index].argmax(-1)
        o_preds = prediction[:,:,o_index]
        preds_final = np.where(o_preds < self.confth, preds_without_o , preds)

        pairs = set()
        processed = []

        # Iterate over document
        for p_doc, token_map, offsets, tokens, doc in zip(
            preds_final, self.ds["token_map"], self.ds["offset_mapping"], self.ds["tokens"], self.ds["document"]
        ):
            # Iterate over sequence
            for p_token, (start_idx, end_idx) in zip(p_doc, offsets):
                label_pred = id2label[p_token]

                if start_idx + end_idx == 0:
                    # [CLS] token i.e. BOS
                    continue

                if token_map[start_idx] == -1:
                    start_idx += 1

                # ignore "\n\n"
                while start_idx < len(token_map) and tokens[token_map[start_idx]].isspace():
                    start_idx += 1

                if start_idx >= len(token_map):
                    break

                token_id = token_map[start_idx]
                pair = (doc, token_id)

                # ignore "O", preds, phone number and  email
                if label_pred in ("O", "B-EMAIL", "B-PHONE_NUM", "I-PHONE_NUM") or token_id == -1:
                    continue

                if pair in pairs:
                    continue

                processed.append(
                    {"document": doc, "token": token_id, "label": label_pred, "token_str": tokens[token_id]}
                )
                pairs.add(pair)

        pred_df = pd.DataFrame(processed + self.emails + self.phone_nums)
        pred_df["row_id"] = list(range(len(pred_df)))

        return pred_df

    def compute_metrics_from_df(self, gt_df, pred_df):
        """
        Compute the LB metric (lb) and other auxiliary metrics
        """

        references = {(row.document, row.token, row.label) for row in gt_df.itertuples()}
        predictions = {(row.document, row.token, row.label) for row in pred_df.itertuples()}

        score_per_type = defaultdict(PRFScore)
        references = set(references)

        for ex in predictions:
            pred_type = ex[-1] # (document, token, label)
            if pred_type != 'O':
                pred_type = pred_type[2:] # avoid B- and I- prefix

            if pred_type not in score_per_type:
                score_per_type[pred_type] = PRFScore()

            if ex in references:
                score_per_type[pred_type].tp += 1
                references.remove(ex)
            else:
                score_per_type[pred_type].fp += 1

        for doc, tok, ref_type in references:
            if ref_type != 'O':
                ref_type = ref_type[2:] # avoid B- and I- prefix

            if ref_type not in score_per_type:
                score_per_type[ref_type] = PRFScore()
            score_per_type[ref_type].fn += 1

        totals = PRFScore()

        for prf in score_per_type.values():
            totals += prf

        return {
            "precision": totals.precision,
            "recall": totals.recall,
            "f5": totals.f5,
            **{
                f"{v_k}-{k}": v_v
                for k in set([l[2:] for l in self.label2id.keys() if l!= 'O'])
                for v_k, v_v in score_per_type[k].to_dict().items()
            },
        }

## Model

In [13]:
class BiLSTMHead(nn.Module):
    def __init__(self, in_features: int, hidden_dim: int, num_layers: int = 1, dropout: float = 0.1):
        super().__init__()
        self.lstm = nn.LSTM(
            in_features,
            hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout,
        )
        self.out_features = hidden_dim

    def forward(self, x: Tensor):
        self.lstm.flatten_parameters()
        h, _ = self.lstm(x)
        h_f, h_b = torch.split(h, h.size(-1)//2, dim=-1)  # split forward and backward
        h = torch.stack([h_f, h_b], dim=0).mean(dim=0)
        return h


class BiLSTMLongFormerForTokenClassification(LongformerForTokenClassification):
    def __init__(self, config):
        super().__init__(config)
        self.bi_lstm = BiLSTMHead(
            in_features=self.config.hidden_size,
            hidden_dim=self.config.hidden_size,
        )
        self.classifier = nn.Sequential(self.bi_lstm, self.classifier)

In [14]:
class ModelInit:
    model_class = BiLSTMLongFormerForTokenClassification

    def __init__(
        self,
        checkpoint: str,
        id2label: dict,
        label2id: dict,
        freeze_embedding: bool,
        freeze_layers: int,
    ) -> None:
        self.model = self.model_class.from_pretrained(
            checkpoint,
            num_labels=len(id2label),
            id2label=id2label,
            label2id=label2id,
            ignore_mismatched_sizes=True
        )
        for param in self.model.base_model.embeddings.parameters():
            param.requires_grad = False if freeze_embedding else True
        for layer in self.model.base_model.encoder.layer[:freeze_layers]:
            for param in layer.parameters():
                param.requires_grad = False
        self.weight = copy.deepcopy(self.model.state_dict())

    def __call__(self) -> model_class:
        self.model.load_state_dict(self.weight)
        return self.model

model_init = ModelInit(
    TRAINING_MODEL_PATH,
    id2label=id2label,
    label2id=label2id,
    freeze_embedding=FREEZE_EMBEDDING,
    freeze_layers=FREEZE_LAYERS,
)

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing BiLSTMLongFormerForTokenClassification: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing BiLSTMLongFormerForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BiLSTMLongFormerForTokenClassification from the checkpoint of a model that you expect to be exactly identica

## Split

In [15]:
# split according to document id
folds = [
    (
        np.array([i for i, d in enumerate(ds["original"]["document"]) if int(d) % N_SPLITS != s]),
        np.array([i for i, d in enumerate(ds["original"]["document"]) if int(d) % N_SPLITS == s])
    )
    for s in range(N_SPLITS)
]

exclude_indices = []
if FILTER_ORIGINAL:
    negative_idxs = [i for i, labels in enumerate(ds["original"]["provided_labels"]) if not any(np.array(labels) != "O")]
    exclude_indices = negative_idxs[len(negative_idxs)//3:]

## Train

In [16]:
for fold_idx, (train_idx, eval_idx) in enumerate(folds):
    args.run_name = f"fold-{fold_idx}"
    args.output_dir = os.path.join(OUTPUT_DIR, f"fold_{fold_idx}")
    if Path(args.output_dir).joinpath("eval_result.json").exists():
        continue
    original_ds = ds["original"].select([i for i in train_idx if i not in exclude_indices])
    train_ds = concatenate_datasets([original_ds, ds["extra"]])
    train_ds = train_ds.map(train_encoder, num_proc=os.cpu_count())
    eval_ds = ds["original"].select(eval_idx)
    eval_ds = eval_ds.map(eval_encoder, num_proc=os.cpu_count())
    trainer = Trainer(
        args=args,
        model_init=model_init,
        train_dataset=train_ds,
        eval_dataset=eval_ds,
        tokenizer=tokenizer,
        compute_metrics=MetricsComputerV2(eval_ds=eval_ds, label2id=label2id),
        data_collator=DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=512),
    )
    trainer.train()
    eval_res = trainer.evaluate(eval_dataset=eval_ds)
    with open(os.path.join(args.output_dir, "eval_result.json"), "w") as f:
        json.dump(eval_res, f)
    del trainer
    gc.collect()
    torch.cuda.empty_cache()
    wandb.finish()

#0:   0%|          | 0/502 [00:00<?, ?ex/s]

#1:   0%|          | 0/502 [00:00<?, ?ex/s]

#2:   0%|          | 0/502 [00:00<?, ?ex/s]

#3:   0%|          | 0/502 [00:00<?, ?ex/s]

#4:   0%|          | 0/502 [00:00<?, ?ex/s]

...

#7:   0%|          | 0/213 [00:00<?, ?ex/s]

Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `BiLSTMLongFormerForTokenClassification.forward` and have been ignored: length, trailing_whitespace, offset_mapping, provided_labels, tokens, token_map, document, full_text. If length, trailing_whitespace, offset_mapping, provided_labels, tokens, token_map, document, full_text are not expected by `BiLSTMLongFormerForTokenClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4025
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Grad

Step,Training Loss,Validation Loss,Precision,Recall,F5,P-username,R-username,F5-username,P-url Personal,R-url Personal,F5-url Personal,P-street Address,R-street Address,F5-street Address,P-email,R-email,F5-email,P-id Num,R-id Num,F5-id Num,P-name Student,R-name Student,F5-name Student,P-phone Num,R-phone Num,F5-phone Num
50,0.648300,0.279197,1.000000,0.034146,0.035464,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
100,0.123600,0.019538,1.000000,0.034146,0.035464,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
150,0.070800,0.007315,0.686567,0.074797,0.077451,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.543478,0.044964,0.046608,1.000000,1.000000,1.000000
200,0.044700,0.005374,0.423671,0.401626,0.402431,0.000000,0.000000,0.000000,0.125000,0.120000,0.120185,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.414498,0.401079,0.401579,1.000000,1.000000,1.000000
250,0.033200,0.004335,0.400821,0.476423,0.472991,0.000000,0.000000,0.000000,0.206522,0.760000,0.688982,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.421667,0.455036,0.453655,1.000000,1.000000,1.000000
300,0.026900,0.003565,0.453804,0.543089,0.539011,0.000000,0.000000,0.000000,0.380952,0.640000,0.623688,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.443284,0.534173,0.529993,1.000000,1.000000,1.000000
350,0.024900,0.003037,0.617564,0.708943,0.704931,0.000000,0.000000,0.000000,0.421053,0.640000,0.627451,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.777778,0.583333,0.588997,0.617323,0.705036,0.701204,1.000000,1.000000,1.000000
400,0.026700,0.002686,0.627877,0.798374,0.790122,0.000000,0.000000,0.000000,0.592593,0.640000,0.638037,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.750000,0.757282,0.616343,0.800360,0.791273,1.000000,1.000000,1.000000
450,0.015700,0.002835,0.624694,0.830894,0.820478,0.000000,0.000000,0.000000,0.283951,0.920000,0.847025,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.733333,0.916667,0.907937,0.653295,0.820144,0.812166,1.000000,1.000000,1.000000
500,0.015800,0.002632,0.660274,0.783740,0.778143,0.000000,0.000000,0.000000,0.619048,0.520000,0.523220,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.666667,0.675325,0.649926,0.791367,0.784798,1.000000,1.000000,1.000000


The following columns in the evaluation set don't have a corresponding argument in `BiLSTMLongFormerForTokenClassification.forward` and have been ignored: length, trailing_whitespace, offset_mapping, provided_labels, tokens, token_map, document, full_text. If length, trailing_whitespace, offset_mapping, provided_labels, tokens, token_map, document, full_text are not expected by `BiLSTMLongFormerForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1706
  Batch size = 8
Saving model checkpoint to output/fold_3/checkpoint-50
Configuration saved in output/fold_3/checkpoint-50/config.json
Model weights saved in output/fold_3/checkpoint-50/pytorch_model.bin
tokenizer config file saved in output/fold_3/checkpoint-50/tokenizer_config.json
Special tokens file saved in output/fold_3/checkpoint-50/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BiLSTMLongFormerForTokenClass

eval/f5,▁▁▁▄▅▅▇▇█▇██████
eval/f5-EMAIL,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/f5-ID_NUM,▁▁▁▁▁▁▆▇█▆▇█████
eval/f5-NAME_STUDENT,▁▁▁▄▅▅▇██▇██████
eval/f5-PHONE_NUM,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/f5-STREET_ADDRESS,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/f5-URL_PERSONAL,▁▁▁▂▆▆▆▆█▅▇██▇██
eval/f5-USERNAME,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/p-EMAIL,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/p-ID_NUM,▁▁▁▁▁▁▆█▆██▆▆▆▆▆


## Log CV

In [17]:
wandb.init(name="cv")
results = dict()
for res_json_path in Path(OUTPUT_DIR).glob("fold*/eval_result.json"):
    fold = res_json_path.parent.name.split("_")[-1]
    with open(res_json_path, "r") as f:
        res = json.load(f)
        results[fold] = {k.replace("eval_", ""): v for k, v in res.items()}
results["cv"] = {key: np.mean([r[key] for r in results.values()]) for key in results["0"].keys()}
table = wandb.Table(columns=["fold"] + list(results["0"].keys()))
for f, res in results.items():
    table.add_data(f, *[res[c] for c in table.columns if c != "fold"])
wandb.log({"eval_result": table})
wandb.finish()